# LangChain's Tool Utilization: Examining Diverse Applications 

* [1. LangChain Agents and Toolkits](#toolkits)
* [2. Google Search](#google)
* [3. Requests](#requests)
* [4. Python REPL](#repl)
* [5. Wikipedia](#wiki)
* [6. Wolfram Alpha](#wolfram)
* [7. A Showcase of Collaboration](#collab)
* [8. Additional Resources](#resources)

In [1]:
import os
from keys import OPENAI_API_KEY, SERPAPI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="toolkits">
    
## 1. LangChain Agents and Toolkits
    
</a>

Tools are modular, reusable components designed to accomplish specific tasks or provide answers to distinct types of questions. Here are several examples of tools in LangChain:
- **Google Search**: uses the Google Search API to fetch relevant information from the web;
- **Requests**: employs the popular Python library "requests" to interact with web services, access APIs, or obtain data from different online sources;
- **Python REPL**(Read-Eval-Print Loop): allows users to execute Python code on-the-fly to perform calculations, manipulate data, or test algorithms. It serves as an interactive programming environment within the LangChain system.
- **Wikipedia**: leverages the Wikipedia API to search and retrieve relevant information from the vast repository of knowledge on Wikipedia;
- **Wolfram Alpha**: uses computational knowledge engine of Wolfram Alpha to answer complex questions, perform advanced calculations, or generate visual representations of data.


In LangChain, an **Agent** is a bot that acts using natural language instructions and can use tools to answer its queries. An agent has access to a suite of tools and can decide which of these tools to call, depending on the user input.  

To create an agent in LangChain, it is possible to use the `initialize_agent` function along with the `load_tools` function to prepare the tools the agent can use. 

Below, is an example of the usage of [SerpApi service](https://serpapi.com/) to fetch Google search results or the Python requests wrapper when required.  

In [2]:
# !pip install langchain==0.0.208 deeplake openai tiktoken
# !pip install google-search-results

In [3]:
from langchain import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = OpenAI(model_name="text-davinci-003", temperature=0)

tools = load_tools(['serpapi', 'requests_all'], 
                   llm=llm, 
                   serpapi_api_key=SERPAPI_API_KEY)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [4]:
# Run the agent
agent.run("Tell me in two sentences, what is midjourney?")



> Entering new AgentExecutor chain...
 Midjourney is a website, so I should look it up online.
Action: Search
Action Input: "midjourney"
Observation: Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc. Midjourney generates images from natural language descriptions, called "prompts", similar to OpenAI's DALL-E and Stable Diffusion.
Thought: I now know the final answer
Final Answer: Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc. It generates images from natural language descriptions, called "prompts".

> Finished chain.


'Midjourney is a generative artificial intelligence program and service created and hosted by San Francisco-based independent research lab Midjourney, Inc. It generates images from natural language descriptions, called "prompts".'

<hr>
<a class="anchor" id="google">
    
## 2. Google Search
    
</a>


LLMs possess knowledge only up to the point at which they were trained, leaving them unaware of any information beyond that timeframe. That is why integrating search engines as tools within the LangChain system brings a significant advantage. 

The LangChain library provides a Google Search API wrapper that can be used either as a **standalone utility or as a tool within an agent**.

In [5]:
import os
from keys import OPENAI_API_KEY, GOOGLE_API_KEY, GOOGLE_CSE_ID

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_CSE_ID"] = GOOGLE_CSE_ID

In [6]:
# Option 1: as a standalone utility
from langchain. utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Portugal?", 3)

[{'title': 'Lisbon, Portugal - Image of the Week - Earth Watching',
  'link': 'https://earth.esa.int/web/earth-watching/image-of-the-week/content/-/article/lisbon-portugal-landsat/',
  'snippet': 'Lisbon is the capital and the largest city of Portugal. It is the westernmost large city located in continental Europe, as well as its westernmost capital\xa0...'},
 {'title': 'Portugal - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Portugal',
  'snippet': 'Lisbon is the capital and largest city by population, being also the main spot for tourists alongside Porto and Algarve. Portuguese Republic. República\xa0...'},
 {'title': 'Where is the Capital of Portugal and What Can I Do There?',
  'link': 'https://www.pousadasofportugal.com/what-is-the-capital-of-portugal-and-what-can-i-do-there/',
  'snippet': 'Lisbon, or Lisboa in Portuguese, is the capital of Portugal, and one of the oldest cities in the world, dated back to 1200 BC.'}]

In [7]:
# Option 2: as a tool within an angent
# (the agent will load the search results and provide them to the LLM to answer a question)

from langchain.llms import OpenAI
from langchain.agents import initialize_agent, load_tools, AgentType

llm = OpenAI(model_name="text-davinci-003", temperature=0) # LLM that will be used by the agent

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

print( agent("What is the national drink in Portugal?") )



> Entering new AgentExecutor chain...
 I should research this online.
Action: google_search
Action Input: "national drink in Portugal"
Observation: Jun 23, 2023 ... But of them all, ginjinha is without a doubt the national drink of Portugal. What drink is Porto famous for? It's right there in the name, Port ... It might even be a surprise to some that its slogan 'O licor de Portugal' claims to be the national liquor. Is this really the national drink of Portugal ... May 12, 2020 ... Many know Port wine, without a doubt, Portugal's most popular fortified wine. But, besides drinking the several varieties of existing Port wine ... Medronho (Aguardente de Medronhos) is a spirit that's made from Medronho berries, which grow wild throughout Portugal especially in the Algarve and Altentejo ... Caipirinha (Portuguese pronunciation: [kajpiˈɾĩɲɐ]) is Brazil's national cocktail, made with cachaça (sugarcane hard liquor), sugar, and lime. The drink is ... Dec 29, 2021 ... Ginjinha or just Ginja 

<hr>
<a class="anchor" id="requests">
    
## 3. Requests
    
</a>

LangChain offers a convenient wrapper built around the Python Requests module to facilitate LLMs with the information available on specific websites.

In the example below, a "fake" RESTful backend is setup using [mockapi.io](https://mockapi.io/):

1. Go to mockapi.io and sign up for a free account;
2. Once signed up, log in to your account;
3. Click on "New Project" (the "+" icon) and give your project a name (no need to fill in any optional fields);
4. Once the project is created, click on it to view the unique API endpoint;
5. Click on "New Resource" to create a new resource for your API. For example, if you want to create an endpoint for users, you can name the resource "users";
6. Define the schema for your resource. For instance, if you want each user to have an id, name, and email, you can use the following schema:

<code>{
    "id": "integer",
    "name": "string",
    "email": "string"
}</code>

In [8]:
# Using the LangChain tools to interact with our fake RESTful backend
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

In [9]:
url = "https://644696c1ee791e1e2903b0bb.mockapi.io/user"
response = agent.run("Get the list of users at "+url+" and tell me the total number of users")



> Entering new AgentExecutor chain...
 I need to get the list of users from the URL
Action: requests_get
Action Input: https://644696c1ee791e1e2903b0bb.mockapi.io/user
Observation: [{"createdAt":"2023-04-24T07:55:47.634Z","name":"Mr. Kelly Balistreri","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/1244.jpg","id":"1"},{"createdAt":"2023-04-24T03:54:44.108Z","name":"Bradley Cronin","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/615.jpg","id":"2"},{"createdAt":"2023-04-24T14:32:29.991Z","name":"Jennifer Block Sr.","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/105.jpg","id":"3"},{"createdAt":"2023-04-23T23:30:05.868Z","name":"Nathaniel Toy V","avatar":"https://cloudflare-ipfs.com/ipfs/Qmd3W5DuhgHirLHGVixi6V76LhCkZUz6pnFt5AJBiyvHye/avatar/533.jpg","id":"4"},{"createdAt":"2023-04-24T03:01:15.797Z","name":"Regina Veum","avatar":"https://cloud

<hr>
<a class="anchor" id="repl">
    
## 4. Python REPL
    
</a>

Python REPL tool allows to execute Python code generated by the LLM (it can be useful for complex calculations since LLMs are not good at solving algorithmic and math problems, but are good at writing code).

In [10]:
# An Example of Python-REPL Tool Usage
tools = load_tools(["python_repl"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
  )

In [11]:
print( agent.run("Create a list of random strings containing 4 letters, list should contain 30 examples, and sort the list alphabetically") )



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I need to generate a list of random strings, sort them, and then print the result
Action: Python_REPL
Action Input: 
import random

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: NameError("name 'string' is not defined")
Thought: I need to import the string module
Action: Python_REPL
Action Input: 
import random
import string

my_list = []
for i in range(30):
    my_list.append(''.join(random.choices(string.ascii_lowercase, k=4)))

my_list.sort()
print(my_list)
Observation: ['amwg', 'boxn', 'ciny', 'coso', 'dfui', 'guoa', 'gzul', 'htqg', 'isxr', 'iweo', 'lrzx', 'mclp', 'mpxn', 'nkuh', 'ohee', 'plpk', 'prmc', 'qvgt', 'raqp', 'shvw', 'skvr', 'twyv', 'ucpm', 'vcds', 'vozp', 'vtkg', 'xpyp', 'xyok', 'xzmw', 'xzvx']

Thought: I now know the final answer
Final Answer: ['amwg', 'boxn', 'ciny', 'coso', 'dfui', 'guoa', 'gzul', 'htqg', 'isxr', 'iweo', 'lrzx', 'mclp', 'mpxn', 'nkuh', 'ohee', 'pl

<hr>
<a class="anchor" id="wiki">
    
## 5. Wikipedia
    
</a>

The Wikipedia API tool in LangChain allows LLMs to interact with the Wikipedia API to fetch information and use it to answer questions. 

In [13]:
#!pip install Wikipedia

In [14]:
tools = load_tools(["wikipedia"])

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True  
)

In [15]:
print(agent.run("What is Zelenskyy known for"))



> Entering new AgentExecutor chain...
 I should look up information about Zelenskyy
Action: Wikipedia
Action Input: Volodymyr Zelenskyy
Observation: Page: Volodymyr Zelenskyy
Summary: Volodymyr Oleksandrovych Zelenskyy (also romanized as Zelensky or Zelenskiy; born 25 January 1978) is a Ukrainian politician and former comedian and actor who is the sixth and current president of Ukraine since 2019.
Born to a Ukrainian Jewish family, Zelenskyy grew up as a native Russian speaker in Kryvyi Rih, a major city of Dnipropetrovsk Oblast in central Ukraine. Prior to his acting career, he obtained a degree in law from the Kyiv National Economic University. He then pursued a career in comedy and created the production company Kvartal 95, which produced films, cartoons, and TV shows including the TV series Servant of the People, in which Zelenskyy played a fictional Ukrainian president. The series aired from 2015 to 2019 and was immensely popular. A political party with the same name as the TV s

<hr>
<a class="anchor" id="wolfram">
    
## 6. Wolfram Alpha
    
</a>

<hr>
<a class="anchor" id="collab">
    
## 7. A Showcase of Collaboration
    
</a>

<hr>
<a class="anchor" id="resources">
    
## 8. Additional Resources
    
</a>

- [SerpApi service](https://serpapi.com/)
- [How to get Search Engine ID](https://help.elfsight.com/article/331-how-to-get-search-engine-id)
- [Setting up Google API keys](https://support.google.com/googleapi/answer/6158862?hl=en)